# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import mplcursors
from tkinter import *
from tkinter import ttk, filedialog
from tkinter.filedialog import askopenfile

%matplotlib inline

%matplotlib widget

%config InlineBackend.figure_format = 'retina'

plt.style.use('seaborn-whitegrid')


## Functions

In [2]:
def split_finance_data(financeData, date1, date2):
    transactionTypes = financeData['Type'].unique()   
    
    mask = (financeData['Date'] >= date1) & (financeData['Date'] <= date2)
    croppedFinanceData = financeData.loc[mask]

    cashflow = {"Cashout" : {},
                "Cashin" : {}
            }

    for count,type in enumerate(transactionTypes):
        croppedByType = croppedFinanceData[croppedFinanceData['Type'] == type]
        cashout = croppedByType['Value'][croppedByType['Value'] < 0].sum()
        cashin = croppedByType['Value'][croppedByType['Value'] > 0].sum()
        cashflow["Cashout"][transactionTypes[count]] = abs(cashout)
        cashflow["Cashin"][transactionTypes[count]] = cashin

    print(cashflow)

def plot_balance(financeData, win):
    # #Change this to produce 3 graphs from 1 function
    # fig = Figure(figsize=(4, 15))
    # ax1 = fig.add_subplot(311)
    # balancePlot = ax1.plot(financeData["Date"], financeData["Balance"])
    # ax2 = fig.add_subplot(312)
    # balancePlot = ax2.plot(financeData["Date"], financeData["Balance"])
    # ax3 = fig.add_subplot(313)
    # balancePlot = ax3.plot(financeData["Date"], financeData["Balance"])
    # ax1.tick_params(labelrotation=45)

    
    # canvas = FigureCanvasTkAgg(fig,
    #                            master = win)  

    
    # mplcursors.cursor(balancePlot)
    # canvas.draw()
  
    # canvas.get_tk_widget().grid(row=5, column=0, padx=5, pady=5, columnspan=2)

    canvas=Canvas(win,bg='#FFFFFF',width=300,height=300,scrollregion=(0,0,500,500))
    hbar=Scrollbar(win,orient=HORIZONTAL)
    hbar.pack(side=BOTTOM,fill=X)
    hbar.config(command=canvas.xview)
    vbar=Scrollbar(win,orient=VERTICAL)
    vbar.pack(side=RIGHT,fill=Y)
    vbar.config(command=canvas.yview)
    canvas.config(width=300,height=300)
    canvas.config(xscrollcommand=hbar.set, yscrollcommand=vbar.set)
    canvas.pack(side=LEFT,expand=True,fill=BOTH)

def upload_file(t1):
    global df
    f_types = [('CSV files',"*.csv"),('All',"*.*")]
    file = filedialog.askopenfilename(filetypes=f_types)
    # title.config(text=file) # display the path 
    df=pd.read_csv(file) # create DataFrame
    df['Date'] = pd.to_datetime(df["Date"], format='%d/%m/%Y')
    str1=file
    t1.insert(END, str1) # add to Text widget

## Tkinter

In [3]:

# Creatign window and starting 
win = Tk()
# win.geometry("400x400")  # Size of the window 
win.title('Finance Tracker')
# win.resizable(0,0)
# my_font1=('times', 12, 'bold')

browse = Button(win, width=15, text='Browse File', command = lambda:upload_file(t1))
browse.grid(row=0,column=0,sticky=W, padx=5, pady=5, columnspan=1) 

t1=Text(win,width=35,height=1)
t1.grid(row=0,column=1,sticky=W, padx= 5, columnspan=1)

date1Input = Label(win, text='First Date:')
date1Input.grid(row=1, column=0, columnspan=1)

date1Entry = Entry(win, width=15)
date1Entry.grid(row=1, column=1, sticky=W, padx=5, columnspan=1)

date2Input = Label(win, text='Second Date:')
date2Input.grid(row=2, column=0, pady=5, columnspan=1)

date2Entry = Entry(win, width=15)
date2Entry.grid(row=2, column=1, sticky=W, padx=5, pady=5, columnspan=1)

runOperation = Button(win, width=50, text='Run Data', command= lambda:split_finance_data(df, date1Entry.get(), date2Entry.get()))
runOperation.grid(row=3, column=0, columnspan=2)

plot_button = Button(win, width=50, height = 2, text = "Plot Graphs", command = lambda: plot_balance(df, win))
plot_button.grid(row=4, column=0, padx=5, pady=5, columnspan=2)

win.mainloop()  # Keep the window open


